# PID Speed Control

In [1]:
import nest_asyncio
nest_asyncio.apply()

import asyncio, sys, os
sys.path.append(os.path.join(os.getenv('IOT49'), 'projects/robot/code-pi'))

from robot import LivePlot, BLE_UART, RobotComm, PID_CPT1, PID_CPT2
from bleak import BleakError
from bokeh.io import output_notebook
from struct import pack, unpack
import stm32, time
import numpy as np


output_notebook()

def exception_handler(loop, context):
    msg = context.get("exception", context["message"])
    print("***** asyncio:", context)
    print("***** msg:", msg)
    

class Robot:
    
    def __init__(self, robot_comm, ble_rx_queue):
        self.robot_comm = robot_comm
        self.ble_rx_queue = ble_rx_queue
        self.stop = False
        
    async def ble_rx(self):
        rc = self.robot_comm
        rx_queue = self.ble_rx_queue
        try:
            last_rx = time.monotonic()
            while True:
                data = await asyncio.wait_for(rx_queue.get(), timeout=10.0)
                code, value = unpack('>Bf', data)
                code = chr(code)
                dt = time.monotonic() - last_rx
                last_rx = time.monotonic()
                if code != 'h' and dt > 0.01:
                    print(f"RECV {code} = {value:8.3f}   dt = {1000*dt:12.4} [ms]")                
                if code == '1':
                    kp = self.kp0 + value/10
                    print("kp =", kp)
                    rc.set_kp(self.pid, kp)
                elif code == '2':
                    kp = self.ki0 + value/10
                    print("ki =", ki)
                    rc.set_kp(self.pid, ki)
                elif code == '3':
                    kp = self.kd0 + value/1000
                    print("kd =", kp)
                    rc.set_kp(self.pid, kd)
                elif code == 'c':
                    print("STOP!")
                    self.stop = True
                elif code != 'h' and dt > 0.01:
                    print(f"RECV {code} = {value:8.3f}   dt = {1000*dt:12.4} [ms]")
                elif code == 'q':
                    print("-"*30, "QUIT!")
        except asyncio.CancelledError:
            print("recv cancelled, stop!")
        except asyncio.TimeoutError:
            print("timeout, stop!")
        except Exception as e:
            print("*** recv", e)
                
    async def step_response(self, fs=100, step=50, kp0=4.0, ki0=20, kd0=0.0):
        rc = self.robot_comm
        rc.fs = fs
        self.kp0 = kp0
        self.ki0 = ki0
        self.kd0 = kd0
        y_axis = ['cpt1_sp', 'cpt1', 'duty1']
        lp = LivePlot('time [s]', y_axis, rollover=500, title="Speed Control", y_range=(-120, 120))
        lp.show(rc.get_state_queue())
        for pid in [ PID_CPT1 ]:
            self.pid = pid
            rc.set_kp(pid, kp0)
            rc.set_ki(pid, ki0)
            rc.set_kd(pid, kd0)
        rc.start_controller(fs, 'speed_control')
        while not self.stop:
            for v_sp in [step, 0, -step, step, -step, 0]:
                rc.set_fstate('cpt1_sp', v_sp)
                # rc.set_fstate('cpt2_sp', v_sp)
                await asyncio.sleep(1)
        await asyncio.sleep(0.5)
        await rc.ping()
        rc.stop_controller()
    
   
print("reset")
stm32.hard_reset()
if False:
    import time
    print("rsync")
    stm32.rsync(dry_run=False)
    print("reset again")
    stm32.hard_reset()
    
print("start controller ...")

async with RobotComm(pwm_freq=10_000, baudrate=1_000_000) as rc:
    ble_rx_queue = asyncio.Queue()
    print("connect to remote")
    async with BLE_UART(lambda _, data: ble_rx_queue.put_nowait(data)) as ble:
        
        async def main(controller):
            print("main ...")
            asyncio.get_event_loop().set_exception_handler(exception_handler)
            # handle commands from ble remote controller
            asyncio.create_task(robot.ble_rx())
            # operate the robot
            await controller()
            
        robot = Robot(rc, ble_rx_queue)
        asyncio.run(main(robot.step_response))
        

Loading BokehJS ...

reset
start controller ...
MCU: init controller

connect to remote
found 30:AE:A4:28:39:F2: mpy-uart
main ...


RECV c =    1.000   dt =    1.774e+03 [ms]
STOP!
RECV v =    4.224   dt =    1.585e+03 [ms]
RECV v =    4.224   dt =    1.585e+03 [ms]
timeout, stop!
MCU: File "<stdin>", line 2, in
MCU:  <module>
  Fi
MCU: le "/spi/lib/com


In [7]:
import nest_asyncio
nest_asyncio.apply()

import asyncio, sys, os
sys.path.append(os.path.join(os.getenv('IOT49'), 'projects/robot/code-pi'))

from robot import RobotComm, PID_CPT1, PID_CPT2
from robot import LivePlot
from robot import BLE_UART
from bleak import BleakError
from struct import pack, unpack
import stm32, time
import numpy as np

from bokeh.io import output_notebook

output_notebook()

def exception_handler(loop, context):
    msg = context.get("exception", context["message"])
    print("***** asyncio:", context)
    print("***** msg:", msg)
    
async def step_response(fs, sp, kp, ki, kd):
    async with RobotComm(fs=fs) as rc:
        asyncio.get_event_loop().set_exception_handler(exception_handler)
        y_axis = ['cpt1_sp', 'cpt1', 'duty1']
        lp = LivePlot('time [s]', y_axis, rollover=2000, title="Speed Control", y_range=(-120, 120))
        lp.show(rc.get_state_queue())
        for pid in [ PID_CPT1, PID_CPT2 ]:
            rc.set_kp(pid, kp)
            rc.set_ki(pid, ki)
            rc.set_kd(pid, kd)
        rc.start_controller(fs, 'speed_control')
        for v_sp in [sp, 0, -sp, sp, -sp, 0]:
            rc.set_fstate('cpt1_sp', v_sp)
            # rc.set_fstate('cpt2_sp', v_sp)
            await asyncio.sleep(2)
        await asyncio.sleep(0.5)
        await rc.ping()
        rc.stop_controller()

async def velocity(fs, N, kp, ki, kd):
    async with RobotComm(fs=fs) as rc:
        asyncio.get_event_loop().set_exception_handler(exception_handler)
        y_axis = ['cpt1', 'cpt1_sp', 'duty1']
        lp = LivePlot('cpt1_sp', y_axis, rollover=2000, title="Speed Control")
        lp.show(rc.get_state_queue())
        for pid in [ PID_CPT1, PID_CPT2 ]:
            rc.set_kp(pid, kp)
            rc.set_ki(pid, ki)
            rc.set_kd(pid, kd)
        rc.start_controller(fs, 'speed_control')
        for v_sp in np.linspace(0, 60, N):
            rc.set_fstate('cpt1_sp', v_sp)
            # rc.set_fstate('cpt2_sp', v_sp)
            await asyncio.sleep(0.1)
        await asyncio.sleep(0.5)
        await rc.ping()
        rc.stop_controller()

print("reset")
stm32.hard_reset()
if False:
    import time
    print("rsync")
    stm32.rsync(dry_run=False)
    print("reset again")
    stm32.hard_reset()

# asyncio.run(velocity(fs=100, N=100, kp=4.0, ki=20, kd=0.0))
asyncio.run(step_response(fs=100, sp=50, kp=4.0, ki=20, kd=0.0))


Loading BokehJS ...

reset
MCU: init controller

